# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [22]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-06 06:34:32--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-06 06:34:32 (69.5 MB/s) - ‘iris.csv.1’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [23]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [24]:
data.head().T

,0,1,2,3,4
sepal.length,5.1,4.9,4.7,4.6,5.0
sepal.width,3.5,3.0,3.2,3.1,3.6
petal.length,1.4,1.4,1.3,1.5,1.4
petal.width,0.2,0.2,0.2,0.2,0.2
variety,Setosa,Setosa,Setosa,Setosa,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [25]:
from sklearn.model_selection import train_test_split

data_latih, data_uji = train_test_split(data, test_size = 0.2, random_state=42)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [26]:
print(data_latih.shape[0])
print(data_uji.shape[0])

120
30


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [27]:
label_uji = data_uji.pop('variety')

In [28]:
label_uji.info()

<class 'pandas.core.series.Series'>
Index: 30 entries, 73 to 30
Series name: variety
Non-Null Count  Dtype 
--------------  ----- 
30 non-null     object
dtypes: object(1)
memory usage: 480.0+ bytes


## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [29]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas]=data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas]=data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas]=data_temp
  return trainingset

Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training.

In [30]:
trainingset = buat_trainingset(data_latih)

Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [31]:
print(trainingset)

{'Setosa':      sepal.length  sepal.width  petal.length  petal.width  variety
22            4.6          3.6           1.0          0.2      1.0
15            5.7          4.4           1.5          0.4      1.0
65            6.7          3.1           4.4          1.4     -1.0
11            4.8          3.4           1.6          0.2      1.0
42            4.4          3.2           1.3          0.2      1.0
..            ...          ...           ...          ...      ...
71            6.1          2.8           4.0          1.3     -1.0
106           4.9          2.5           4.5          1.7     -1.0
14            5.8          4.0           1.2          0.2      1.0
92            5.8          2.6           4.0          1.2     -1.0
102           7.1          3.0           5.9          2.1     -1.0

[120 rows x 5 columns], 'Versicolor':      sepal.length  sepal.width  petal.length  petal.width  variety
22            4.6          3.6           1.0          0.2     -1.0
15          

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [32]:
def hitung_cost_gradient(W,X,Y, regularization):
  jarak = 1-(Y*np.dot(X,W))
  dw=np.zeros(len(W))
  if max(0, jarak) == 0:
    di=W
  else:
    di= W-(regularization*Y*X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [33]:
from sklearn.utils import shuffle

def sgd(data_latih, label_latih, learning_rate = 0.000001, max_epoch = 1000, regularization = 10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1, max_epoch):
    X,Y = shuffle(data_latih, label_latih, random_state=101)
    for index, x in enumerate (X):
      delta = hitung_cost_gradient(bobot, x ,Y[index], regularization)
      bobot = bobot - (learning_rate*delta)
    return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [34]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w = {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih,label_latih)
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [35]:
W = training(trainingset)

Tampilkan isi variabel **W**

In [36]:
print(W)

{'Setosa': array([ 0.11299181,  0.4369682 , -0.70294803, -0.3099774 ]), 'Versicolor': array([-0.08598473, -0.20097984,  0.13899087,  0.02699673]), 'Virginica': array([-0.27398034, -0.32097818,  0.38897579,  0.24798573])}


## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [37]:
def testing(W, data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi, y_prediksi)
  return prediksi

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [38]:
def testing_onevsrest(W,data_uji):
  kelas_prediksi = np.array([])
  for i in range(data_uji.shape[0]):
      kelas = None
      max_score = -float('inf')
      for key, value in W.items():
          score = np.dot(value, data_uji.to_numpy()[i])
          if score > max_score:
              max_score = score
              kelas = key
      kelas_prediksi = np.append(kelas_prediksi, kelas)
  return kelas_prediksi

In [39]:
prediksi = testing_onevsrest(W,data_uji)

Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [40]:
print(prediksi)

['Versicolor' 'Setosa' 'Virginica' 'Versicolor' 'Versicolor' 'Setosa'
 'Versicolor' 'Virginica' 'Virginica' 'Versicolor' 'Virginica' 'Setosa'
 'Setosa' 'Setosa' 'Setosa' 'Versicolor' 'Virginica' 'Versicolor'
 'Virginica' 'Virginica' 'Setosa' 'Virginica' 'Setosa' 'Virginica'
 'Virginica' 'Virginica' 'Virginica' 'Virginica' 'Setosa' 'Setosa']


In [41]:
print('Prediksi benar: ',sum(prediksi==label_uji))

Prediksi benar:  28
